In [2]:
import json 
import numpy as np 
import os,sys 

In [3]:
root_path="/home/user/xsj/LLMTaskPlanning/alfred_multiagent/data/json_2.1.0/"
root_path_len=len(root_path)
# filepaths=[]
# for root, dirs,files in os.walk(root_path):
#     for filename in files:
#         if "json" in filename:
#             filepaths.append(os.path.join(root,filename)[root_path_len:])
#         # print(root,dirs,files)
# # filepaths

In [4]:
# scene_dict={}
# for trail_path in filepaths:
#     with open(os.path.join(root_path,trail_path),'r') as f :
#         config=json.load(f)
#     if not config["scene"]["floor_plan"] in scene_dict.keys():
#         scene_dict[config["scene"]["floor_plan"]]=[]    
#     scene_dict[config["scene"]["floor_plan"]].append(trail_path)
        

In [5]:
# with open(os.path.join(root_path,'../../scene_dict.json'),'w') as f :
#     f.write(json.dumps(scene_dict,sort_keys=False,indent=4,separators=(',',':')))
with open(os.path.join(root_path,'../../scene_dict.json'),'r') as f :
    scene_dict=json.load(f)

In [6]:
scene_dict.keys()

dict_keys(['FloorPlan9', 'FloorPlan315', 'FloorPlan29', 'FloorPlan215', 'FloorPlan226', 'FloorPlan404', 'FloorPlan425', 'FloorPlan325', 'FloorPlan30', 'FloorPlan1', 'FloorPlan8', 'FloorPlan14', 'FloorPlan26', 'FloorPlan206', 'FloorPlan329', 'FloorPlan7', 'FloorPlan413', 'FloorPlan20', 'FloorPlan19', 'FloorPlan2', 'FloorPlan27', 'FloorPlan11', 'FloorPlan6', 'FloorPlan320', 'FloorPlan321', 'FloorPlan330', 'FloorPlan21', 'FloorPlan418', 'FloorPlan203', 'FloorPlan18', 'FloorPlan427', 'FloorPlan209', 'FloorPlan326', 'FloorPlan414', 'FloorPlan211', 'FloorPlan213', 'FloorPlan23', 'FloorPlan25', 'FloorPlan4', 'FloorPlan5', 'FloorPlan301', 'FloorPlan314', 'FloorPlan426', 'FloorPlan309', 'FloorPlan216', 'FloorPlan225', 'FloorPlan401', 'FloorPlan423', 'FloorPlan422', 'FloorPlan16', 'FloorPlan217', 'FloorPlan319', 'FloorPlan24', 'FloorPlan310', 'FloorPlan222', 'FloorPlan403', 'FloorPlan15', 'FloorPlan12', 'FloorPlan402', 'FloorPlan415', 'FloorPlan428', 'FloorPlan13', 'FloorPlan302', 'FloorPlan312'

In [7]:
from ai2thor.controller import Controller

In [ ]:
for scene_name in scene_dict.keys():
    task_config={}
    task_config["scene_name"]=scene_name
    task_config["objects"]={}
    task_config["turk_annotations"]={}
    controller = Controller(
        agentMode="arm",
        agentCount=2,
        visibilityDistance=1.5,
        scene=scene_name,

        # step sizes
        # gridSize=self.gridSize,
        snapToGrid=True,
        rotateStepDegrees=90,

        # image modalities
        renderDepthImage=False,
        renderInstanceSegmentation=False,

        # camera properties
        # width=player_screen_width,
        # height=player_screen_height,
        # x_display=x_display,
        fieldOfView=90)
    multiEvents = controller.step(action="GetReachablePositions")
    print(len(multiEvents.metadata["objects"]))
    all_object_name_in_ori_scene=set()
    for obj in multiEvents.metadata["objects"]:
        task_config["objects"][obj["objectId"]]={"name":obj["name"],"objectType":obj["objectType"]}
        all_object_name_in_ori_scene.add(obj["name"])
    tasks=scene_dict[scene_name]
    for trial_path in tasks:
        with open(os.path.join(root_path,trial_path),'r') as f:
            trial=json.load(f)
        # check 1, if all objects in the trial are in the original scene 
        check_pass=True
        for object_poses in trial["scene"]["object_poses"]:
            if not object_poses["objectName"] in all_object_name_in_ori_scene:
                check_pass=False
                break
        if check_pass:
            task_config["turk_annotations"][trial_path]=trial["turk_annotations"]
            print(scene_name,trial_path)
    with open(os.path.join(root_path,'../../multiagent_longtasks/{}.json'.format(scene_name)),'w') as f :
        f.write(json.dumps(task_config,sort_keys=False,indent=4,separators=(',',':')))
    

In [ ]:
same_floor=scene_dict["FloorPlan325"]

In [ ]:
# object pose in the floor 
object_pose_dict={}
object_name='Vase_afb1a602'
for trial_path in same_floor:
    with open(os.path.join(root_path,trial_path),'r') as f:
        trial=json.load(f)
    for obj in trial['scene']['object_poses']:
        if obj["objectName"] == object_name:
            object_pose_dict[trial_path]=obj
object_pose_dict

In [ ]:
same_floor_object_name={}
for trial_path in same_floor:
    with open(os.path.join(root_path,trial_path),'r') as f:
        trial=json.load(f)
    for obj in trial['scene']['object_poses']:
        if not obj['objectName'] in same_floor_object_name.keys(): 
            same_floor_object_name[obj['objectName']]={}
        if not trial_path in same_floor_object_name[obj['objectName']].keys():
            same_floor_object_name[obj['objectName']][trial_path]=0
        same_floor_object_name[obj['objectName']][trial_path]+=1
same_floor_object_name